# From `intro_to_DESI_EDR_files.ipynb`

In [1]:
# import some helpful python packages 
import os
import numpy as np

from astropy.io import fits
from astropy.table import Table
from astropy.convolution import convolve, Gaussian1DKernel

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('../mpl/desi.mplstyle')

In [2]:
# import DESI related modules - 
from desimodel.footprint import radec2pix      # For getting healpix values
import desispec.io                             # Input/Output functions related to DESI spectra
from desispec import coaddition                # Functions related to coadding the spectra

# DESI targeting masks - 
from desitarget.sv1 import sv1_targetmask    # For SV1
from desitarget.sv2 import sv2_targetmask    # For SV2
from desitarget.sv3 import sv3_targetmask    # For SV3

In [3]:
# Release directory path

specprod = 'iron'    # Internal name for the EDR
specprod_dir = '/global/cfs/cdirs/desi/spectro/redux/' + specprod
print(specprod_dir)

/global/cfs/cdirs/desi/spectro/redux/iron


In [4]:
# List everything in this directory
os.listdir(specprod_dir)

['healpix',
 'tiles',
 'exposures-iron.csv',
 'run',
 'calibnight',
 'exposures',
 'tiles-iron.fits',
 'processing_tables',
 'preproc',
 'inventory-iron.txt',
 'exposure_tables',
 'redux_iron.sha256sum',
 'exposures-iron.fits',
 'tiles-iron.csv',
 'zcatalog']

In [9]:
exp_table = Table.read(f'{specprod_dir}/exposures-{specprod}.fits', hdu='EXPOSURES')
print(f"Tiles table columns: {exp_table.colnames}")

Tiles table columns: ['NIGHT', 'EXPID', 'TILEID', 'TILERA', 'TILEDEC', 'MJD', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'EXPTIME', 'EFFTIME_SPEC', 'GOALTIME', 'GOALTYPE', 'MINTFRAC', 'AIRMASS', 'EBV', 'SEEING_ETC', 'EFFTIME_ETC', 'TSNR2_ELG', 'TSNR2_QSO', 'TSNR2_LRG', 'TSNR2_LYA', 'TSNR2_BGS', 'TSNR2_GPBDARK', 'TSNR2_GPBBRIGHT', 'TSNR2_GPBBACKUP', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK', 'GPB_EFFTIME_DARK', 'GPB_EFFTIME_BRIGHT', 'GPB_EFFTIME_BACKUP', 'TRANSPARENCY_GFA', 'SEEING_GFA', 'FIBER_FRACFLUX_GFA', 'FIBER_FRACFLUX_ELG_GFA', 'FIBER_FRACFLUX_BGS_GFA', 'FIBERFAC_GFA', 'FIBERFAC_ELG_GFA', 'FIBERFAC_BGS_GFA', 'AIRMASS_GFA', 'SKY_MAG_AB_GFA', 'SKY_MAG_G_SPEC', 'SKY_MAG_R_SPEC', 'SKY_MAG_Z_SPEC', 'EFFTIME_GFA', 'EFFTIME_DARK_GFA', 'EFFTIME_BRIGHT_GFA', 'EFFTIME_BACKUP_GFA']


In [10]:
exp_table[-5:]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes16,bytes19,float64,float64,float64,bytes6,float64,float32,float64,float64,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210529,90247,373,245.832,43.335,59364.36430717,sv3,bright,bright,sv3bright,1117.044921875,337.6169128417969,220.0,bright,0.9,1.068032,0.009423913434147835,1.4692250490188599,223.98949,44.475147,8.6310425,27.963772,13.32962854905054,2411.5493,2913.2263,571.0539,3861.4192,344.7333,382.4863,337.6169,152.48772523755858,252.18994,299.93616,283.0008,1.033268169456845,1.5989511114379837,0.3463963950802347,0.2711160875582591,0.13095087611297396,0.5744627241541538,0.6178762888349207,0.6495438397604425,1.0794371013994377,20.525505155668775,20.297341392770107,20.449208280154785,19.76493542876646,302.50769913715436,253.50730817203686,302.50769913715436,273.4625058258671
20210529,90248,400,252.442,34.39,59364.37940271,sv3,bright,bright,sv3bright,939.0352783203125,324.3939208984375,220.0,bright,0.9,1.048001,0.02507576160132885,1.29488205909729,223.24913,42.884575,8.598594,26.822409,13.778553664730861,2317.0994,2856.3599,555.2721,3844.6543,330.66275,368.80737,324.39392,157.62331993475487,247.26717,291.64703,281.7721,1.0140228921898826,1.457591698451905,0.40184047540259993,0.30849427371325466,0.14702371651459745,0.6728532105171252,0.7109799191994934,0.7364581942095623,1.0582610372774308,20.471386857019684,20.238753596063795,20.384506599404546,19.781617847518657,290.8780128434468,247.4650862587995,290.8780128434468,288.78444084042695
20210529,90249,427,269.538,65.929,59364.39286466,sv3,bright,bright,sv3bright,638.1410522460938,305.1478576660156,220.0,bright,0.9,1.210807,0.03996047377586365,1.233667016029358,222.87892,40.401855,9.118238,25.374088,12.912202950241044,2179.6274,2848.7534,533.6405,4032.643,312.80807,347.45596,305.14786,147.71247739144215,246.60869,280.28543,295.54968,1.0477574055550516,1.08992566816822,0.5676842869416672,0.41038684882798,0.18894336064826067,0.9630449341670511,0.9581835389545319,0.9601785625345641,1.213112847568369,20.346008656681235,20.16165043687451,20.307942291873115,19.595014353283528,282.96405063110075,247.82334776412873,282.96405063110075,360.81216722111327
20210529,90250,565,269.926,62.441,59364.40219283,sv3,bright,bright,sv3bright,523.5750122070312,301.34124755859375,220.0,bright,0.9,1.170556,0.031614355742931366,1.1474930047988892,222.65558,40.03024,9.407354,24.999666,15.261678844550625,2152.4375,2863.223,528.574,4095.7031,308.19223,344.26007,301.34125,174.58991312083205,247.86128,277.62433,300.1713,1.0429984187389545,0.9969984126737262,0.6200876372740602,0.44224438682694484,0.20191861148396323,1.0563439920369584,1.0352732075685347,1.0277561394185621,1.173014299261516,20.36560251288922,20.187495833922565,20.329918150146984,19.650058927549892,275.4071756801791,242.9241353658027,275.4071756801791,374.1056713204288
20210610,93262,521,212.891,-0.679,59376.20555175,sv3,dark,dark,sv3dark,1378.7406005859375,1226.9554443359375,1200.0,dark,0.9,1.203036,0.0666235014796257,1.0793989896774292,1211.8282,142.66924,35.086723,104.27875,94.79930891506373,8636.729,14916.666,2550.5806,14568.3

Let's use this to find the number of exposures in each `SURVEY` and each `PROGRAM`.

In [11]:
for survey in ['cmx', 'sv1', 'sv2', 'sv3']:
    print(f'{survey}: Nexps={np.sum(exp_table["SURVEY"]==survey)}')

cmx: Nexps=4
sv1: Nexps=1673
sv2: Nexps=72
sv3: Nexps=710


In [12]:
for program in ['bright', 'dark']:
    print(f'{program}: Nexps={np.sum(exp_table["PROGRAM"]==program)}')

bright: Nexps=897
dark: Nexps=1194


## Redshift Catalogs

The redshift catalogs that are in the `zcatalog/` directory are divided into `zpix` and `ztile` files. The `zpix` files have the format: `zpix-{survey}-{program}.fits`, while the `ztile` have the format: `ztile-{survey}-{program}-{group}.fits`. The datamodel for the different files within this folder can be found [here](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/index.html). Each of these files contain redshift and targeting information of the sources based on how their spectra are coadded. 

* **survey**: SURVEY that the target was observed in. This can be sv1, sv2, or main.
* **program**: FIBER ASSIGNMENT PROGRAM. This is the planned observing conditions for the target. It can be dark or bright or backup. In case of cmx and sv1, there is 'other' as well.
* **group**: This denotes what type of coadd. It can be perexp, pernight, or cumulative.

For example, `zpix-sv1-dark.fits` contains information about the targets that were observed in SV1 for the _dark_ program. Since it is healpix based, the resulting spectra for a particular target is the coadded spectra across all tiles that were observed within the given `SURVEY` and `PROGRAM`. This will provide the highest S/N version of each target.

Note that some of the targets were observed in multiple surveys and/or programs. In such cases, one may want to select the _PRIMARY_ spectrum of a given object. This information is available in the redshift summary catalogs, which will be explained in detail below.

In [5]:
# Listing all the available redshift catalogs

os.listdir(f'{specprod_dir}/zcatalog')

['ztile-special-dark-cumulative.fits',
 'zpix-main-dark.fits',
 'zpix-sv2-backup.fits',
 'zpix-sv3-bright.fits',
 'ztile-special-dark-pernight.fits',
 'zpix-special-backup.fits',
 'ztile-sv3-bright-pernight.fits',
 'ztile-sv1-bright-pernight.fits',
 'ztile-sv2-bright-pernight.fits',
 'zpix-sv3-dark.fits',
 'zpix-sv3-backup.fits',
 'ztile-main-dark-cumulative.fits',
 'ztile-sv1-dark-cumulative.fits',
 'v1',
 'zpix-sv1-bright.fits',
 'ztile-cmx-other-cumulative.fits',
 'zpix-cmx-other.fits',
 'ztile-special-backup-cumulative.fits',
 'zpix-sv2-dark.fits',
 'zpix-sv1-dark.fits',
 'ztile-sv3-backup-cumulative.fits',
 'ztile-special-bright-cumulative.fits',
 'ztile-sv1-other-cumulative.fits',
 'ztile-cmx-other-pernight.fits',
 'ztile-sv2-dark-cumulative.fits',
 'ztile-sv3-bright-cumulative.fits',
 'zpix-main-backup.fits',
 'ztile-sv2-dark-pernight.fits',
 'zall-pix-iron.fits',
 'ztile-special-other-cumulative.fits',
 'ztile-sv3-backup-pernight.fits',
 'ztile-sv1-dark-pernight.fits',
 'v0',
 

Now, we will look at the summary redshift catalogs that contain the `PRIMARY` spectra information

<a class="anchor" id="summary_zcatalog"></a>
## Summary Redshift Catalogs

There are two summary redshift catalogs: <br/>
**zall-pix-fuji.fits** <br/> 
This is a stack of all the healpix-based redshift catalogs, including all surveys and programs. The datamodel is [here](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zall-pix-SPECPROD.html). <br/>
**zall-tilecumulative-fuji.fits** <br/>
This is a stack of all the tile-based cumulative redshift catalogs, including all surveys and programs. The datamodel is [here](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zall-tilecumulative-SPECPROD.html).

For both these summary catalogs, all the columns in the original catalogs ([zpix-SURVEY-PROGRAM.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zpix-SURVEY-PROGRAM.html) and [ztile-SURVEY-PROGRAM-GROUPTYPE.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zpix-SURVEY-PROGRAM.html)) are included. <br/>
Four new columns are added:

1. SV_NSPEC - Number of SV spectra available of each target.
2. SV_PRIMARY - PRIMARY flag for the sources with SV spectra.
3. ZCAT_NSPEC - Number of spectra available in the entire catalog of each target.
4. ZCAT_PRIMARY - PRIMARY flag for the sources in the entire catalog.

### Tile-based vs healpix-based
The choice of which you use will depend on your specific science case. If you want the hightest S/N spectra then you'll likely want to use healpix based redshifts. If you want a more consistant observing depth among the redshfits in your sample, or you need objects that fit on a single pointing, then tile redshifts may be of more 
interest.

We will focus here on the healpix based redshifts.

### Working with the zall-pix file

In [6]:
zpix_cat = Table.read(f'{specprod_dir}/zcatalog/zall-pix-{specprod}.fits', hdu="ZCATALOG")

In [7]:
print(zpix_cat.columns)

<TableColumns names=('TARGETID','SURVEY','PROGRAM','HEALPIX','SPGRPVAL','Z','ZERR','ZWARN','CHI2','COEFF','NPIXELS','SPECTYPE','SUBTYPE','NCOEFF','DELTACHI2','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','PMRA','PMDEC','REF_EPOCH','FA_TARGET','FA_TYPE','OBJTYPE','SUBPRIORITY','OBSCONDITIONS','RELEASE','BRICKNAME','BRICKID','BRICK_OBJID','MORPHTYPE','EBV','FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_IVAR_G','FLUX_IVAR_R','FLUX_IVAR_Z','FLUX_IVAR_W1','FLUX_IVAR_W2','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z','MASKBITS','SERSIC','SHAPE_R','SHAPE_E1','SHAPE_E2','REF_ID','REF_CAT','GAIA_PHOT_G_MEAN_MAG','GAIA_PHOT_BP_MEAN_MAG','GAIA_PHOT_RP_MEAN_MAG','PARALLAX','PHOTSYS','PRIORITY_INIT','NUMOBS_INIT','CMX_TARGET','DESI_TARGET','BGS_TARGET','MWS_TARGET','SCND_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_MWS_TARGET','SV1_SCND_TARGET','SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_MWS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGE

In [17]:
# Targetting bits are coded in the sv*_targetmask.desi_mask
## The long list of targeting reflects the complexity of the targeting process and helps in subselecting from the main target types.

sv1_targetmask.desi_mask

sv1_desi_mask:
  - [LRG,              0, "LRG", {'obsconditions': 'DARK', 'priorities': {'UNOBS': 3200, 'DONE': 2, 'OBS': 1, 'DONOTOBSERVE': 0, 'MORE_ZWARN': 3200, 'MORE_ZGOOD': 3200}, 'numobs': 2}]
  - [ELG,              1, "ELG", {'obsconditions': 'DARK|GRAY', 'priorities': {'UNOBS': 3000, 'DONE': 2, 'OBS': 1, 'DONOTOBSERVE': 0, 'MORE_ZWARN': 3000, 'MORE_ZGOOD': 3000}, 'numobs': 1}]
  - [QSO,              2, "QSO", {'obsconditions': 'DARK', 'priorities': {'UNOBS': 3400, 'MORE_ZGOOD': 3350, 'MORE_ZWARN': 3300, 'MORE_MIDZQSO': 100, 'DONE': 2, 'OBS': 1, 'DONOTOBSERVE': 0}, 'numobs': 4}]
  - [LRG_OPT,          3, "LRG from baseline version of optical cuts", {'obsconditions': 'DARK', 'priorities': {'UNOBS': 3200, 'DONE': 2, 'OBS': 1, 'DONOTOBSERVE': 0, 'MORE_ZWARN': 3200, 'MORE_ZGOOD': 3200}, 'numobs': 4}]
  - [LRG_IR,           4, "LRG from baseline version of IR cuts", {'obsconditions': 'DARK', 'priorities': {'UNOBS': 3200, 'DONE': 2, 'OBS': 1, 'DONOTOBSERVE': 0, 'MORE_ZWARN': 3200, 'MO

In [19]:
# A simple example of using the bitmasks

print(sv1_targetmask.desi_mask.mask("STD_FAINT"), sv1_targetmask.desi_mask.mask("STD_BRIGHT"), sv1_targetmask.desi_mask.mask("STD_FAINT|STD_BRIGHT"))
print(2**33, 2**35, 2**33 + 2**35)

8589934592 34359738368 42949672960
8589934592 34359738368 42949672960


In [7]:
# Targeting information about the DESI targetting is stored in the different desi_target columns
# sv1_targetmask.desi_mask corresponds to SV1_DESI_TARGET
# sv2_targetmask.desi_mask corresponds to SV2_DESI_TARGET
# sv3_targetmask.desi_mask corresponds to SV3_DESI_TARGET

sv1_desi_tgt = zpix_cat['SV1_DESI_TARGET']
sv2_desi_tgt = zpix_cat['SV2_DESI_TARGET']
sv3_desi_tgt = zpix_cat['SV3_DESI_TARGET']

sv1_desi_mask = sv1_targetmask.desi_mask
sv2_desi_mask = sv2_targetmask.desi_mask
sv3_desi_mask = sv3_targetmask.desi_mask

In [30]:
print(type(sv1_desi_tgt), sv1_desi_tgt.shape, type(sv1_desi_mask), sv1_desi_mask[0])

<class 'astropy.table.column.Column'> (28425963,) <class 'desiutil.bitmask.BitMask'> LRG              bit 0 mask 0x1 - LRG


In [38]:
# Selecting candidates - 
# The code below selects the individual targets observed in all the SV1, SV2, and SV3 tiles.

## All BGS targets from sv1, sv2, and sv3
is_bgs = (sv1_desi_tgt & sv1_desi_mask['BGS_ANY'] != 0)|(sv2_desi_tgt & sv2_desi_mask['BGS_ANY'] != 0)|(sv3_desi_tgt & sv3_desi_mask['BGS_ANY'] != 0)
# Number of sources of each target type
n_bgs = len(zpix_cat[is_bgs])

<class 'numpy.ndarray'>


In [39]:
print(type(is_bgs), len(is_bgs))

<class 'numpy.ndarray'> 28425963


In [59]:
demo = zpix_cat[is_bgs][:10]

In [60]:
demo

TARGETID,SURVEY,PROGRAM,HEALPIX,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,FA_TARGET,FA_TYPE,OBJTYPE,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV1_SCND_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV2_SCND_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,PLATE_RA,PLATE_DEC,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,MAIN_NSPEC,MAIN_PRIMARY,SV_NSPEC,SV_PRIMARY,ZCAT_NSPEC,ZCAT_PRIMARY
int64,bytes7,bytes6,int32,int32,float64,float64,int64,float64,float64[10],int64,bytes6,bytes20,int64,float64,int32,float64,float64,float32,float32,float32,int64,uint8,bytes3,float64,int32,int16,bytes8,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,bool,int32,bool,int64,bool
39633487573027812,sv1,backup,7543,7543,0.932752477574551,2.232146096148817e-46,518,9.000000000000002e+99,0.0 .. 0.0,0,GALAXY,--,10,0.0,512,177.09914973580632,69.78972872458297,5.965992,-2.154366,2021.2224,3458764513820540928,1,TGT,0.3483189078040533,7,9011,1767p697,641456,3044,PSF,0.012502538,4.3591485,31.002502,43.928177,24.098131,11.013512,62.345085,12.311493,1.1904991,1.3637573,0.700737,3.3859954,24.081385,34.121483,26.84848,59.82547,72.801956,2072,0.0,0.0,0.0,0.0,1061605590746555136,G2,19.491877,0.0,0.0,0.37880346,N,1100,1,0,0,0,0,0,3458764513820540928,2570,17,0,0,0,0,0,0,0,0,0,177.09914973580632,69.78972872458297,0,0.0,0,0,-4.832,4.832,2.913,2.913,177.04627452349018,0.0,69.77719183580702,0.0,0.0,63.12422,0.022767447,62.131706,8.577588,679.0401,58.29815,0.30570373,0.78199416,3083.46,7.3631525,3014.004,0.005302651,3390.9707,2754.8167,1.0958098,31.583372,9.824189e-06,37.87404,9.624132e-06,0.0,7971.7783,8.87042e-06,3.8165112,50.320282,3146.5842,45.25996,3076.1355,8.5828905,12041.789,2813.1147,5.2180247,82.68565,0,False,1,True,2,False
39633491716998359,sv1,backup,7548,7548,1.1740819206448814,0.00035175786828599233,0,8632.353412371129,5.788790109033574 .. 0.0,7898,QSO,LOZ,4,1842.4068622030318,0,173.6457110782341,70.18888380176844,-0.3064713,0.27902663,2021.1458,8070463726390095957,1,TGT,0.9211791781789317,63,9011,1737p702,642444,1239,PSF,0.0121016195,20.215437,42.178715,41.719902,139.09836,275.7249,234.62234,65.40025,74.67821,1.9517388,0.46013427,

In [62]:
is_confident = demo["DELTACHI2"] > 40
not_confident = demo["DELTACHI2"] < 40
demo[is_confident]

TARGETID,SURVEY,PROGRAM,HEALPIX,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,FA_TARGET,FA_TYPE,OBJTYPE,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV1_SCND_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV2_SCND_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,PLATE_RA,PLATE_DEC,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,MAIN_NSPEC,MAIN_PRIMARY,SV_NSPEC,SV_PRIMARY,ZCAT_NSPEC,ZCAT_PRIMARY
int64,bytes7,bytes6,int32,int32,float64,float64,int64,float64,float64[10],int64,bytes6,bytes20,int64,float64,int32,float64,float64,float32,float32,float32,int64,uint8,bytes3,float64,int32,int16,bytes8,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,bool,int32,bool,int64,bool
39633491716998359,sv1,backup,7548,7548,1.1740819206448814,0.00035175786828599233,0,8632.353412371129,5.788790109033574 .. 0.0,7898,QSO,LOZ,4,1842.4068622030318,0,173.6457110782341,70.18888380176844,-0.3064713,0.27902663,2021.1458,8070463726390095957,1,TGT,0.9211791781789317,63,9011,1737p702,642444,1239,PSF,0.0121016195,20.215437,42.178715,41.719902,139.09836,275.7249,234.62234,65.40025,74.67821,1.9517388,0.46013427,15.745045,32.851418,32.49407,15.745045,32.851418,32.49407,0,0.0,0.0,0.0,0.0,1062379951875295360,G2,19.102213,19.583513,18.449755,-0.04324165,N,3400,4,0,0,0,0,0,8070463726390095957,514,17,34359738368,0,0,0,0,0,0,0,0,173.6457110782341,70.18888380176844,1,300.0676,1,1,1.076,1.5266516,0.899,1.2685527,173.65702234731725,121.03362,70.18551136320335,12.086399,0.3945,42.223923,0.025779344,25.618155,17.108335,476.94568,53.369972,0.49042434,0.5239035,2921.9712,6.7164135,1802.7675,0.009285177,2068.491,3430.5852,1.467554,20.170584,6.0156076e-06,32.230896,3.6826752e-06,0.0,4460.9062,7.341647e-06,4.5912666,30.58489,2964.195,38.973087,1828.3856,17.11762,7006.343,3483.955,6.549245,51.279377,0,False,1,True,2,True
39633495806445941,sv1,backup,7549,7549,1.2668087697077974,0.00020546227726553123,0,10996.49230222404,8.908973492325844 .. 0.0,7746,QSO,LOZ,4,3164.282698497176,0,179.03046966196396,70.86639298857585,0.4734316,-0.33995137,2021.2224,8070503308806111236,1,TGT,0.168378151054291,63,9011,1788p707,643419,2421,PSF,0.012782482,16.248775,26.652971,22.833

In [66]:
is_confident.astype(int)  # flag of confidence -> Δχ^2>40 or not

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 1])